In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import numpy as np
import sys
sys.path.extend(['../commons'])
import dl_commons as dlc
import pandas as pd
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import viz_commons as viz

Using TensorFlow backend.


In [3]:
width = None
pd.options.display.max_rows = 600
pd.options.display.max_columns = width
pd.options.display.max_colwidth = 600
pd.options.display.width = width
pd.options.display.max_seq_items = None
pd.options.display.expand_frame_repr = False
pd.options.display.colheader_justify = 'left'

In [4]:
runs_pub = (
    '/zpool_3TB/i2l/tb_metrics/2017-12-16 18-51-38 PST pLambda=0_score_88.86/test_runs/step_00132200_score88.19_publish/',
    '/zpool_3TB/i2l/tb_metrics/2017-12-21 02-20-10 PST 140K_score_89.0/test_runs/step_00168100_score89.0_publish/',
    '/zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/test_runs/step_00167526_score89.0_publish/'
)
runs_pub_eval = viz.EvalRuns(runs_pub, hyper_names=['raw_data_dir', 'pLambda', 'build_init_model', 'REGROUP_IMAGE', 'adam_alpha', 'adam_beta1', 'adam_beta2'], metric_names=['select_max@id%testing/.*/bleu2', 'selected%testing/.*/edit_distance', 'min@id%testing/.*/edit_distance', 'max@id%testing/.*/bleu$'])
df_pub = runs_pub_eval.df
df_pub

Loaded /zpool_3TB/i2l/tb_metrics/2017-12-16 18-51-38 PST pLambda=0_score_88.86/test_runs/step_00132200_score88.19_publish/store/hyper.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-21 02-20-10 PST 140K_score_89.0/test_runs/step_00168100_score89.0_publish/store/hyper.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/test_runs/step_00167526_score89.0_publish/store/hyper.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-16 18-51-38 PST pLambda=0_score_88.86/test_runs/step_00132200_score88.19_publish/store/args.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-21 02-20-10 PST 140K_score_89.0/test_runs/step_00168100_score89.0_publish/store/args.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/test_runs/step_00167526_score89.0_publish/store/args.pkl
processing file /zpool_3TB/i2l/tb_metrics/2017-12-16 18-51-38 PST pLambda=0_score_88.86/test_runs/step_00132200_score88.19_publish/events.out.tfevents.1515204218.DeepLearner
processin

,raw_data_dir,pLambda,build_init_model,REGROUP_IMAGE,adam_alpha,adam_beta1,adam_beta2,select_max@id%testing/top_1/bleu2,selected%testing/top_1/edit_distance,min@id%testing/top_1/edit_distance,max@id%testing/top_1/bleu
/zpool_3TB/i2l/tb_metrics/2017-12-16 18-51-38 PST pLambda=0_score_88.86/test_runs/step_00132200_score88.19_publish/,../data/dataset3/training_56,0.0,True,"[4, 1]",0.0001,0.5,0.9,0.8819 @ 115675,0.0725 @ 115675,0.0725 @ 115675,0.8784 @ 115675
/zpool_3TB/i2l/tb_metrics/2017-12-21 02-20-10 PST 140K_score_89.0/test_runs/step_00168100_score89.0_publish/,../data/dataset5/training_56,0.0,True,"[4, 1]",0.0001,0.5,0.9,0.8900 @ 147087,0.0671 @ 147087,0.0671 @ 147087,0.8865 @ 147087
/zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/test_runs/step_00167526_score89.0_publish/,../data/dataset5/training_56,0.0,True,None,0.0001,0.5,0.9,0.8900 @ 146585,0.0676 @ 146585,0.0676 @ 146585,0.8869 @ 146585


In [5]:
runs = (
    '/zpool_3TB/i2l/tb_metrics/2017-12-06 15-40-46 PST score_86.14',
    '/zpool_3TB/i2l/tb_metrics/2017-12-07 23-23-25 PST score_87.45',
    '/zpool_3TB/i2l/tb_metrics/2017-12-10 12-52-23 PST score_88.17',
    '/zpool_3TB/i2l/tb_metrics/2017-12-11 22-57-42 PST noInitModel_score_86.54',
    '/zpool_3TB/i2l/tb_metrics/2017-12-12 19-49-24 PST noInitModel_score_88.10',
    '/zpool_3TB/i2l/tb_metrics/2017-12-16 18-51-38 PST pLambda=0_score_88.86',
    '/zpool_3TB/i2l/tb_metrics/2017-12-21 02-20-10 PST 140K_score_89.0',
    '/zpool_3TB/i2l/tb_metrics/2017-12-23 22-02-35 PST 140K_noInitModel_score_88.74',
    '/zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09',
#     '/zpool_3TB/i2l/tb_metrics/2017-12-28 13-46-23 PST 140K_noRegroup_noInitModel_beta2=0.6_score85.96',
    '/zpool_3TB/i2l/tb_metrics/2017-12-31 11-32-42 PST 140K_noRegroup_noInitModel_beta2=0.90_beta1=0.25_score88.05/00169865_beta2=0.90_beta1=0.1_score88.20'
)
runs = viz.EvalRuns(runs, hyper_names=['raw_data_dir', 'pLambda', 'build_init_model', 'REGROUP_IMAGE', 'adam_alpha', 'adam_beta1', 'adam_beta2'], metric_names=['max%u_step', 'select_max@id%validation/.*bleu2', 'selected%edit_distance', 'min@id%edit_distance', 'max@id%validation/.*bleu$'])
df = runs.df

Loaded /zpool_3TB/i2l/tb_metrics/2017-12-06 15-40-46 PST score_86.14/store/hyper.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-07 23-23-25 PST score_87.45/store/hyper.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-10 12-52-23 PST score_88.17/store/hyper.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-11 22-57-42 PST noInitModel_score_86.54/store/hyper.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-12 19-49-24 PST noInitModel_score_88.10/store/hyper.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-16 18-51-38 PST pLambda=0_score_88.86/store/hyper.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-21 02-20-10 PST 140K_score_89.0/store/hyper.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-23 22-02-35 PST 140K_noInitModel_score_88.74/store/hyper.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/store/hyper.pkl
Loaded /zpool_3TB/i2l/tb_metrics/2017-12-31 11-32-42 PST 140K_noRegroup_noInitModel_beta2=0.90_beta1=0.25_score88.05/00169865_beta2=0.90_beta1=0.1_score88.20/store/hy

In [6]:
df[df.raw_data_dir.str.contains('dataset3')]

,raw_data_dir,pLambda,build_init_model,REGROUP_IMAGE,adam_alpha,adam_beta1,adam_beta2,max%u_step,select_max@id%validation/top_1/bleu2,selected%validation/top_1/edit_distance,min@id%validation/top_1/edit_distance,max@id%validation/top_1/bleu
/zpool_3TB/i2l/tb_metrics/2017-12-06 15-40-46 PST score_86.14,../data/dataset3/training_56/,0.0005,undefined,"[4, 1]",0.0001,undefined,undefined,127900,0.8614 @ 87150,0.0847 @ 87150,0.0846 @ 98700,0.8643 @ 87675
/zpool_3TB/i2l/tb_metrics/2017-12-07 23-23-25 PST score_87.45,../data/dataset3/training_56/,0.0005,True,"[4, 1]",0.0001,undefined,undefined,171500,0.8745 @ 129850,0.0744 @ 129850,0.0744 @ 129850,0.8757 @ 129850
/zpool_3TB/i2l/tb_metrics/2017-12-10 12-52-23 PST score_88.17,../data/dataset3/training_56/,0.0005,True,"[4, 1]",0.0001,0.5,0.9,145800,0.8817 @ 106925,0.0765 @ 106925,0.0730 @ 112525,0.8774 @ 112525
/zpool_3TB/i2l/tb_metrics/2017-12-11 22-57-42 PST noInitModel_score_86.54,../data/dataset3/training_56/,0.0005,False,"[4, 1]",0.0001,0.5,0.9,74531,0.8654 @ 63525,0.0818 @ 63525,0.0803 @ 58450,0.8664 @ 58450
/zpool_3TB/i2l/tb_metrics/2017-12-12 19-49-24 PST noInitModel_score_88.10,../data/dataset3/training_56/,0.0005,False,"[4, 1]",0.0001,0.5,0.9,243735,0.8810 @ 179812,0.0750 @ 179812,0.0738 @ 181212,0.8745 @ 181212
/zpool_3TB/i2l/tb_metrics/2017-12-16 18-51-38 PST pLambda=0_score_88.86,../data/dataset3/training_56,0.0000,True,"[4, 1]",0.0001,0.5,0.9,165700,0.8886 @ 115675,0.0688 @ 115675,0.0687 @ 133875,0.8825 @ 133875


In [7]:
df[df.raw_data_dir.str.contains('dataset5')]
# df

,raw_data_dir,pLambda,build_init_model,REGROUP_IMAGE,adam_alpha,adam_beta1,adam_beta2,max%u_step,select_max@id%validation/top_1/bleu2,selected%validation/top_1/edit_distance,min@id%validation/top_1/edit_distance,max@id%validation/top_1/bleu
/zpool_3TB/i2l/tb_metrics/2017-12-21 02-20-10 PST 140K_score_89.0,../data/dataset5/training_56,0.0,True,"[4, 1]",0.0001,0.5,0.9,213000,0.8900 @ 147087,0.0677 @ 147087,0.0669 @ 139737,0.8868 @ 177187
/zpool_3TB/i2l/tb_metrics/2017-12-23 22-02-35 PST 140K_noInitModel_score_88.74,../data/dataset5/training_56,0.0,False,"[4, 1]",0.0001,0.5,0.9,153224,0.8874 @ 126921,0.0691 @ 126921,0.0689 @ 123346,0.8834 @ 126921
/zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09,../data/dataset5/training_56,0.0,True,None,0.0001,0.5,0.9,213965,0.8909 @ 146585,0.0684 @ 146585,0.0675 @ 126921,0.8867 @ 166249
/zpool_3TB/i2l/tb_metrics/2017-12-31 11-32-42 PST 140K_noRegroup_noInitModel_beta2=0.90_beta1=0.25_score88.05/00169865_beta2=0.90_beta1=0.1_score88.20,../data/dataset5/training_56,0.0,False,None,0.0001,0.1,0.9,242500,0.8820 @ 188300,0.0738 @ 188300,0.0726 @ 152687,0.8783 @ 152687


In [8]:
# runs.hypers['/zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09']

In [9]:
reader = viz.TBSummaryReader('/zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09', 56)
df = reader.read('(validation|training)/.*(bleu|edit_distance|ctc_ed|logloss_mean)')

processing file /zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/events.out.tfevents.1514264671.DeepLearner
processing file /zpool_3TB/i2l/tb_metrics/2017-12-25 21-04-15 PST 140K_noRegroup_score89.09/events.out.tfevents.1514403018.DeepLearner
(0.935183048248,
0.935526967049)
Overwriting values
(0.0377222783864,
0.0372374206781)
Overwriting values
(0.930749177933,
0.92874121666)
Overwriting values
(0.0201514195651,
0.0210598260164)
Overwriting values
(0.934884011745,
0.937058806419)
Overwriting values
(0.0379119962454,
0.0367913581431)
Overwriting values
(0.930735349655,
0.93242752552)
Overwriting values
(0.0189689286053,
0.0160191170871)
Overwriting values
(0.936959147453,
0.938866198063)
Overwriting values
(0.0365783199668,
0.0353783741593)
Overwriting values
(0.931754052639,
0.935901582241)
Overwriting values
(0.0176305696368,
0.0164821781218)
Overwriting values
(0.937074780464,
0.939397275448)
Overwriting values
(0.0367513969541,
0.0352781638503)
Overwriti

In [10]:
df

,training/bleu,training/bleu2,u_step,wall_time,training/ctc_ed,training/logloss_mean,validation/top_1/bleu2,validation/top_1/edit_distance,validation/top_1/bleu
87,0.195683,0.027584,100,1.514265e+09,0.699871,4.061313,NaN,NaN,NaN
175,0.192471,0.030383,200,1.514265e+09,0.692371,3.802582,NaN,NaN,NaN
262,0.175016,0.010950,300,1.514265e+09,0.693630,3.599373,NaN,NaN,NaN
350,0.179717,0.047608,400,1.514265e+09,0.680253,3.292958,NaN,NaN,NaN
437,0.186381,0.067868,500,1.514265e+09,0.674165,3.131105,NaN,NaN,NaN
525,0.202425,0.068310,600,1.514265e+09,0.673159,3.061558,NaN,NaN,NaN
612,0.201365,0.076629,700,1.514265e+09,0.658328,2.947630,NaN,NaN,NaN
700,0.202361,0.084264,800,1.514265e+09,0.648076,2.878944,NaN,NaN,NaN
787,0.208640,0.093091,900,1.514265e+09,0.637413,2.810229,NaN,NaN,NaN
875,0.202997,0.104403,1000,1.514266e+09,0.624547,2.743017,NaN,NaN,NaN


In [11]:
l = [1,2,3]
l + [4]

[1, 2, 3, 4]